In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [10]:
# input placeholder
X = tf.placeholder(tf.float32, [None,784])
X_img = tf.reshape(X,[-1,28,28,1])
Y = tf.placeholder(tf.float32, [None,10])

# L1 ImgIn Shape
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
L1 = tf.nn.conv2d(X_img,W1,strides=[1,1,1,1],padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
 # Convolutional Layer #1
conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3, 3],
                         padding="SAME", activation=tf.nn.relu)
# Pooling Layer #1
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                                padding="SAME", strides=2)
dropout1 = tf.layers.dropout(inputs=pool1, rate=0.3)

# Convolutional Layer #2 and Pooling Layer #2
conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3, 3],
                         padding="SAME", activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],
                                padding="SAME", strides=2)
dropout2 = tf.layers.dropout(inputs=pool2, rate=0.3)

# Convolutional Layer #3 and Pooling Layer #3
conv3 = tf.layers.conv2d(inputs=dropout2, filters=128, kernel_size=[3, 3],
                         padding="SAME", activation=tf.nn.relu)
pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2],
                                padding="SAME", strides=2)
dropout3 = tf.layers.dropout(inputs=pool3, rate=0.3)
print(conv1)
print(pool1)
print(dropout1)
print()
print(conv2)
print(pool2)
print(dropout2)
print()
print(conv3)
print(pool3)
print(dropout3)

Tensor("conv2d_8/Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("max_pooling2d_8/MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)
Tensor("dropout_6/Identity:0", shape=(?, 14, 14, 32), dtype=float32)

Tensor("conv2d_9/Relu:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("max_pooling2d_9/MaxPool:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("dropout_7/Identity:0", shape=(?, 7, 7, 64), dtype=float32)

Tensor("conv2d_10/Relu:0", shape=(?, 7, 7, 128), dtype=float32)
Tensor("max_pooling2d_10/MaxPool:0", shape=(?, 4, 4, 128), dtype=float32)
Tensor("dropout_8/Identity:0", shape=(?, 4, 4, 128), dtype=float32)


In [5]:
W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev=0.01))

L2 = tf.nn.conv2d(L1,W2,strides=[1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L2 = tf.reshape(L2, [-1,7*7*64])

In [6]:
W3 = tf.get_variable("W3", shape=[7*7*64,10], initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2,W3)+ b

# define cost/Loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [7]:
# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# train my model
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _, = sess.run([cost,optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    print(" epoch= " , epoch , " cost = ", avg_cost)

 epoch=  0  cost =  0.38429862317036495
 epoch=  1  cost =  0.10877844260158868
 epoch=  2  cost =  0.07813117024065414
 epoch=  3  cost =  0.06380424485203214
 epoch=  4  cost =  0.05484643476507209
 epoch=  5  cost =  0.046676928797228824
 epoch=  6  cost =  0.04049316865053369
 epoch=  7  cost =  0.0363359424437989
 epoch=  8  cost =  0.031473671550100515
 epoch=  9  cost =  0.030353987175886575
 epoch=  10  cost =  0.024791943896667702
 epoch=  11  cost =  0.022865070666514584
 epoch=  12  cost =  0.02161110466377896
 epoch=  13  cost =  0.01772594536046118
 epoch=  14  cost =  0.01688656804731264


AttributeError: module 'tensorflow' has no attribute 'eqaul'

In [10]:
# test model
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("accuracy = " , sess.run(accuracy, feed_dict={X: mnist.test.images, Y:mnist.test.labels}))

accuracy =  0.9859
